# Step 1: Data Ingestion & Preparation

In [1]:
# import the libraries
print("Importing the libraries is in progress...")

import os
import csv
import pandas as pd
import numpy as np
import collections
import socket
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline
import matplotlib.pyplot as plt
import glob
import urllib

print("Import complete successfully!")

Importing the libraries is in progress...
Import complete successfully!


## Data Ingestion

In [2]:
# data preparation step 1
# put into PdM_telemetry.csv for 3. and 4. columns the model and age columns from PdM_machines.csv

print('Data preparation step 1 in progress...')

inp1 = open('./data/00_sourceData/PdM_telemetry.csv', 'r', newline='')
reader1 = csv.reader(inp1, delimiter=',')

inp2 = open('./data/00_sourceData/PdM_machines.csv', 'r', newline='')
reader2 = list(csv.reader(inp2, delimiter=','))

with open('./data/01_intermediateData/PdM_01_telemetry_model_age.csv', 'w', newline='') as out1:
    writer1 = csv.writer(out1)
    for row1 in reader1:
        if row1[0] == "datetime":
            writer1.writerow([row1[0]]+[row1[1]]+["model"]+["age"]+[row1[2]]+[row1[3]]+[row1[4]]+[row1[5]])
        else:
            for row2 in reader2:
                if row2[0] == "machineID":
                    continue
                elif row2[0] == row1[1]:
                    writer1.writerow([row1[0]]+[row1[1]]+[row2[1]]+[row2[2]]+[row1[2]]+[row1[3]]+[row1[4]]+[row1[5]])
                    
inp1.close()
inp2.close()
out1.close()

print('Writing of PdM_01_telemetry_model_age.csv completed successfully.')
print('Data preparation step 1 complete.')

Data preparation step 1 in progress...
Writing of PdM_01_telemetry_model_age.csv completed successfully.
Data preparation step 1 complete.


In [3]:
# data preparation step 2
# put into PdM_01_telemetry_model_age.csv for last column the comp column from PdM_maint.csv

print('Data preparation step 2 in progress...')

inp3 = open('./data/01_intermediateData/PdM_01_telemetry_model_age.csv', 'r', newline='')
reader3 = csv.reader(inp3, delimiter=',')

inp4 = open('./data/00_sourceData/PdM_maint.csv', 'r', newline='')
reader4 = list(csv.reader(inp4, delimiter=','))

with open('./data/01_intermediateData/PdM_02_telemetry_model_age_maint_pre.csv', 'w', newline='') as out2:
    writer2 = csv.writer(out2)
    for row3 in reader3:
        if row3[0] == "datetime":
            writer2.writerow(row3+["maintained_component"])
        else:
            compExist = 0
            for row4 in reader4:
                if row4[0] == row3[0]:
                    if row4[1] == row3[1]:
                        writer2.writerow(row3+[row4[2]])
                        compExist = 1
            else:
                if compExist == 0:
                    writer2.writerow(row3+["-"])
                    
inp3.close()
inp4.close()
out2.close()

print('Writing of PdM_02_telemetry_model_age_maint_pre.csv completed successfully.')
print('Data preparation step 2 complete.')

Data preparation step 2 in progress...
Writing of PdM_02_telemetry_model_age_maint_pre.csv completed successfully.
Data preparation step 2 complete.


In [4]:
# data preparation step 3
# combine (concatenate) data from multiple rows in CSV into one cell in another CSV in PdM_telemetry_model_age_02_maint_pre.csv for last column

print('Data preparation step 3 in progress...')

d_telem = collections.defaultdict(set)

with open("./data/01_intermediateData/PdM_02_telemetry_model_age_maint_pre.csv", "r") as f_input3:
    csv_input3 = csv.reader(f_input3, skipinitialspace=True)
    headers = next(csv_input3)

    for row in csv_input3:
        d_telem[row[0]+","+row[1]+","+row[2]+","+row[3]+","+row[4]+","+row[5]+","+row[6]+","+row[7]].add(row[8])
        #d_telem[row[0]].append(row[1])   # if a list is preferred

with open("./data/01_intermediateData/PdM_03_telemetry_model_age_maint_final.csv", "w") as f_output2:
    csv_output2 = csv.writer(f_output2)
    csv_output2.writerow(headers)

    telem_data = d_telem.keys()

    for telem in telem_data:
        l_maint = list(d_telem[telem])
        csv_output2.writerow([telem, "+".join(l_maint)])

f_input3.close()
f_output2.close()

# Jelen allas szerint a PdM_03_telemetry_model_age_maint_final.csv-bol manualisan el kell tavolitani az osszes "-et es az osszes ures sort (gyakorlatban: \r),
# es akkor lesz jo a fajl.

print('Writing of PdM_03_telemetry_model_age_maint_final.csv completed successfully.')
print('Data preparation step 3 complete.')

Data preparation step 3 in progress...
Writing of PdM_03_telemetry_model_age_maint_final.csv completed successfully.
Data preparation step 3 complete.


In [5]:
# data preparation step 4 skipped because we don't need error data only failure data
# put into PdM_03_telemetry_model_age_maint_final.csv for last column the errorID column from PdM_errors.csv

print ('Data preparation step 4 skipped because we do not need error data only failure data.')
# print('Data preparation step 4 in progress...')

#inp5 = open('./data/01_intermediateData/PdM_03_telemetry_model_age_maint_final.csv', 'r', newline='')
#reader5 = csv.reader(inp5, delimiter=',')

#inp6 = open('./data/00_sourceData/PdM_errors.csv', 'r', newline='')
#reader6 = list(csv.reader(inp6, delimiter=','))

#with open('./data/01_intermediateData/PdM_04_telemetry_model_age_maint_err_pre.csv', 'w', newline='') as out3:
#    writer3 = csv.writer(out3)
#    for row5 in reader5:
#        if row5[0] == "datetime":
#            writer3.writerow(row5+["errors_failures"])
#        else:
#            errExist = 0
#            for row6 in reader6:
#                if row6[0] == row5[0]:
#                    if row6[1] == row5[1]:
#                        writer3.writerow(row5+[row6[2][-1:]])
#                        errExist = 1
#            else:
#                if errExist == 0:
#                    writer3.writerow(row5+["0"])
                    
#inp5.close()
#inp6.close()
#out3.close()

#print('Writing of PdM_04_telemetry_model_age_maint_err_pre.csv completed successfully.')
#print('Data preparation step 4 complete.')

Data preparation step 4 skipped because we do not need error data.


In [7]:
# data preparation step 5
# put into PdM_telemetry_model_age_05_maint_final.csv to the last column (called failures) the coded value of failure column from PdM_failures.csv

print('Data preparation step 5 in progress...')

inp7 = open('./data/01_intermediateData/PdM_03_telemetry_model_age_maint_final.csv', 'r', newline='')
reader7 = csv.reader(inp7, delimiter=',')

inp8 = open('./data/00_sourceData/PdM_failures.csv', 'r', newline='')
reader8 = list(csv.reader(inp8, delimiter=','))

with open('./data/01_intermediateData/PdM_05_telemetry_model_age_maint_fail.csv', 'w', newline='') as out4:
    writer4 = csv.writer(out4)
    for row7 in reader7:
        if row7[0] == "datetime":
            writer4.writerow(row7+["failures"])
        else:
            failExist = 0
            for row8 in reader8:
                if row8[0] == row7[0]:
                    if row8[1] == row7[1]:
                        writer4.writerow(row7+[row8[2][-1:]+"0"])
                        failExist = 1
            else:
                if failExist == 0:
                    writer4.writerow(row7+["0"])
                    
inp7.close()
inp8.close()
out4.close()

print('Writing of PdM_05_telemetry_model_age_maint_fail.csv completed successfully.')
print('Data preparation step 5 complete.')

Data preparation step 5 in progress...
Writing of PdM_05_telemetry_model_age_maint_fail.csv completed successfully.
Data preparation step 5 complete.


In [8]:
# Sorting input file by machineID (ascending) and datetime (descending) to determine RUL
print('Sorting data is in progress...')

df1 = pd.read_csv('./data/01_intermediateData/PdM_05_telemetry_model_age_maint_fail.csv', delimiter=',')
df1 = df1.sort_values(['machineID', 'datetime'], ascending=[True, False])
df1.to_csv('./data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv',index=False)

print('Writing of .../data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv completed successfully.')

Sorting data is in progress...
Writing of .../data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv completed successfully.


In [9]:
# Determine RUL and break down the data by error type and save them in separate csv files
print('Determine RUL is in progress...')

# failure1
print('failure1 RUL determination is in progress...')
with open("data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv", "r", newline='') as source910:
    reader910 = csv.reader(source910)
    with open("data/01_intermediateData/PdM_07_RUL10.csv", "w", newline='') as result910:
        writer510 = csv.writer(result910)
        writeData = 0
        mIDprev = 1
        mID = 1
        i = 0
        for r910 in reader910:
            if r910[0] == "datetime":
                writer510.writerow(r910+["RUL"])
            else:
                mID = int(r910[1])
                if mID != mIDprev:
                    writeData = 0
                    mIDprev = mID
                else:
                    if r910[9] == "10":
                        writeData = 1
                        i = 0
                        writer510.writerow(r910+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r910[9] == "0":
                        writer510.writerow(r910+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r910[9] != "0" and r910[9] != "10":
                        writer510.writerow([r910[0]]+[r910[1]]+[r910[2]]+[r910[3]]+[r910[4]]+[r910[5]]+[r910[6]]+[r910[7]]+[r910[8]]+["0"]+[i])
                        mIDprev = mID
                        i = i+1
source910.close()
result910.close()

# failure2
print('failure2 RUL determination is in progress...')
with open("data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv", "r", newline='') as source920:
    reader920 = csv.reader(source920)
    with open("data/01_intermediateData/PdM_07_RUL20.csv", "w", newline='') as result920:
        writer520 = csv.writer(result920)
        writeData = 0
        mIDprev = 1
        mID = 1
        i = 0
        for r920 in reader920:
            if r920[0] == "datetime":
                writer520.writerow(r920+["RUL"])
            else:
                mID = int(r920[1])
                if mID != mIDprev:
                    writeData = 0
                    mIDprev = mID
                else:
                    if r920[9] == "20":
                        writeData = 1
                        i = 0
                        writer520.writerow(r920+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r920[9] == "0":
                        writer520.writerow(r920+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r920[9] != "0" and r920[9] != "20":
                        writer520.writerow([r920[0]]+[r920[1]]+[r920[2]]+[r920[3]]+[r920[4]]+[r920[5]]+[r920[6]]+[r920[7]]+[r920[8]]+["0"]+[i])
                        mIDprev = mID
                        i = i+1
source920.close()
result920.close()

# failure3
print('failure3 RUL determination is in progress...')
with open("data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv", "r", newline='') as source930:
    reader930 = csv.reader(source930)
    with open("data/01_intermediateData/PdM_07_RUL30.csv", "w", newline='') as result930:
        writer530 = csv.writer(result930)
        writeData = 0
        mIDprev = 1
        mID = 1
        i = 0
        for r930 in reader930:
            if r930[0] == "datetime":
                writer530.writerow(r930+["RUL"])
            else:
                mID = int(r930[1])
                if mID != mIDprev:
                    writeData = 0
                    mIDprev = mID
                else:
                    if r930[9] == "30":
                        writeData = 1
                        i = 0
                        writer530.writerow(r930+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r930[9] == "0":
                        writer530.writerow(r930+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r930[9] != "0" and r930[9] != "30":
                        writer530.writerow([r930[0]]+[r930[1]]+[r930[2]]+[r930[3]]+[r930[4]]+[r930[5]]+[r930[6]]+[r930[7]]+[r930[8]]+["0"]+[i])
                        mIDprev = mID
                        i = i+1
source930.close()
result930.close()

# failure4
print('failure4 RUL determination is in progress...')
with open("data/01_intermediateData/PdM_06_telemetry_model_age_maint_fail_final_sorted.csv", "r", newline='') as source940:
    reader940 = csv.reader(source940)
    with open("data/01_intermediateData/PdM_07_RUL40.csv", "w", newline='') as result940:
        writer540 = csv.writer(result940)
        writeData = 0
        mIDprev = 1
        mID = 1
        i = 0
        for r940 in reader940:
            if r940[0] == "datetime":
                writer540.writerow(r940+["RUL"])
            else:
                mID = int(r940[1])
                if mID != mIDprev:
                    writeData = 0
                    mIDprev = mID
                else:
                    if r940[9] == "40":
                        writeData = 1
                        i = 0
                        writer540.writerow(r940+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r940[9] == "0":
                        writer540.writerow(r940+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1 and r940[9] != "0" and r940[9] != "40":
                        writer540.writerow([r940[0]]+[r940[1]]+[r940[2]]+[r940[3]]+[r940[4]]+[r940[5]]+[r940[6]]+[r940[7]]+[r940[8]]+["0"]+[i])
                        mIDprev = mID
                        i = i+1
source940.close()
result940.close()

print('Writing of .../data/01_intermediateData/PdM_07_RULn.csv files completed successfully.')

Determine RUL is in progress...
failure1 RUL determination is in progress...
failure2 RUL determination is in progress...
failure3 RUL determination is in progress...
failure4 RUL determination is in progress...
Writing of .../data/01_intermediateData/PdM_07_RULn.csv files completed successfully.


In [10]:
# Sorting intermediate file by machineID (ascending) and datetime (ascending)
print('Sorting data is in progress...')

# failure1
df310 = pd.read_csv('./data/01_intermediateData/PdM_07_RUL10.csv', delimiter=',')
df310 = df310.sort_values(['machineID', 'datetime'], ascending=[True, True])
df310.to_csv('./data/01_intermediateData/PdM_08_RUL10_sorted_asc.csv',index=False)
# failure2
df320 = pd.read_csv('./data/01_intermediateData/PdM_07_RUL20.csv', delimiter=',')
df320 = df320.sort_values(['machineID', 'datetime'], ascending=[True, True])
df320.to_csv('./data/01_intermediateData/PdM_08_RUL20_sorted_asc.csv',index=False)
# failure3
df330 = pd.read_csv('./data/01_intermediateData/PdM_07_RUL30.csv', delimiter=',')
df330 = df330.sort_values(['machineID', 'datetime'], ascending=[True, True])
df330.to_csv('./data/01_intermediateData/PdM_08_RUL30_sorted_asc.csv',index=False)
# failure4
df340 = pd.read_csv('./data/01_intermediateData/PdM_07_RUL40.csv', delimiter=',')
df340 = df340.sort_values(['machineID', 'datetime'], ascending=[True, True])
df340.to_csv('./data/01_intermediateData/PdM_08_RUL40_sorted_asc.csv',index=False)

print('Writing of .../data/01_intermediateData/PdM_08_RULn_sorted_asc.csv files completed successfully.')

Sorting data is in progress...
Writing of .../data/01_intermediateData/PdM_08_RULn_sorted_asc.csv files completed successfully.


In [11]:
# Determine cycle
print('Determine cycle is in progress...')

# failure1
print('failure1 cycle determination is in progress...')
with open("data/01_intermediateData/PdM_08_RUL10_sorted_asc.csv", "r", newline='') as source410:
    reader410 = csv.reader(source410)
    with open("data/01_intermediateData/PdM_09_RUL_cycle_10.csv", "w", newline='') as result410:
        writer410 = csv.writer(result410)
        mIDprev = 1
        mID = 1
        i = 1
        prevRUL = 0
        for r410 in reader410:
            if r410[0] == "datetime":
                writer410.writerow((r410[0], r410[1], str('cycle'), r410[2], r410[3], r410[4], r410[5], r410[6], r410[7], r410[8], r410[9], r410[10]))
            else:
                mID = int(r410[1])
                if mID != mIDprev:
                    i = 1
                    writer410.writerow((r410[0], r410[1], str(i), r410[2][-1:], r410[3], r410[4], r410[5], r410[6], r410[7], r410[8], r410[9], r410[10]))
                    mIDprev = mID
                    i = i+1
                    prevRUL = r410[10]
                else:
                    if r410[9] != str(0):
                        writer410.writerow((r410[0], r410[1], str(i), r410[2][-1:], r410[3], r410[4], r410[5], r410[6], r410[7], r410[8], r410[9], r410[10]))
                        prevRUL = r410[10]
                    elif r410[9] == str(0) and prevRUL == str(0):
                        i = 1
                        writer410.writerow((r410[0], r410[1], str(i), r410[2][-1:], r410[3], r410[4], r410[5], r410[6], r410[7], r410[8], r410[9], r410[10]))
                        prevRUL = r410[10]
                        i = i+1
                    else:
                        writer410.writerow((r410[0], r410[1], str(i), r410[2][-1:], r410[3], r410[4], r410[5], r410[6], r410[7], r410[8], r410[9], r410[10]))
                        i = i+1
                        prevRUL = r410[10]
source410.close()
result410.close()

# failure2
print('failure2 cycle determination is in progress...')
with open("data/01_intermediateData/PdM_08_RUL20_sorted_asc.csv", "r", newline='') as source420:
    reader420 = csv.reader(source420)
    with open("data/01_intermediateData/PdM_09_RUL_cycle_20.csv", "w", newline='') as result420:
        writer420 = csv.writer(result420)
        mIDprev = 1
        mID = 1
        i = 1
        prevRUL = 0
        for r420 in reader420:
            if r420[0] == "datetime":
                writer420.writerow((r420[0], r420[1], str('cycle'), r420[2], r420[3], r420[4], r420[5], r420[6], r420[7], r420[8], r420[9], r420[10]))
            else:
                mID = int(r420[1])
                if mID != mIDprev:
                    i = 1
                    writer420.writerow((r420[0], r420[1], str(i), r420[2][-1:], r420[3], r420[4], r420[5], r420[6], r420[7], r420[8], r420[9], r420[10]))
                    mIDprev = mID
                    i = i+1
                    prevRUL = r420[10]
                else:
                    if r420[9] != str(0):
                        writer420.writerow((r420[0], r420[1], str(i), r420[2][-1:], r420[3], r420[4], r420[5], r420[6], r420[7], r420[8], r420[9], r420[10]))
                        prevRUL = r420[10]
                    elif r420[9] == str(0) and prevRUL == str(0):
                        i = 1
                        writer420.writerow((r420[0], r420[1], str(i), r420[2][-1:], r420[3], r420[4], r420[5], r420[6], r420[7], r420[8], r420[9], r420[10]))
                        prevRUL = r420[10]
                        i = i+1
                    else:
                        writer420.writerow((r420[0], r420[1], str(i), r420[2][-1:], r420[3], r420[4], r420[5], r420[6], r420[7], r420[8], r420[9], r420[10]))
                        i = i+1
                        prevRUL = r420[10]
source420.close()
result420.close()

# failure3
print('failure3 cycle determination is in progress...')
with open("data/01_intermediateData/PdM_08_RUL30_sorted_asc.csv", "r", newline='') as source430:
    reader430 = csv.reader(source430)
    with open("data/01_intermediateData/PdM_09_RUL_cycle_30.csv", "w", newline='') as result430:
        writer430 = csv.writer(result430)
        mIDprev = 1
        mID = 1
        i = 1
        prevRUL = 0
        for r430 in reader430:
            if r430[0] == "datetime":
                writer430.writerow((r430[0], r430[1], str('cycle'), r430[2], r430[3], r430[4], r430[5], r430[6], r430[7], r430[8], r430[9], r430[10]))
            else:
                mID = int(r430[1])
                if mID != mIDprev:
                    i = 1
                    writer430.writerow((r430[0], r430[1], str(i), r430[2][-1:], r430[3], r430[4], r430[5], r430[6], r430[7], r430[8], r430[9], r430[10]))
                    mIDprev = mID
                    i = i+1
                    prevRUL = r430[10]
                else:
                    if r430[9] != str(0):
                        writer430.writerow((r430[0], r430[1], str(i), r430[2][-1:], r430[3], r430[4], r430[5], r430[6], r430[7], r430[8], r430[9], r430[10]))
                        prevRUL = r430[10]
                    elif r430[9] == str(0) and prevRUL == str(0):
                        i = 1
                        writer430.writerow((r430[0], r430[1], str(i), r430[2][-1:], r430[3], r430[4], r430[5], r430[6], r430[7], r430[8], r430[9], r430[10]))
                        prevRUL = r430[10]
                        i = i+1
                    else:
                        writer430.writerow((r430[0], r430[1], str(i), r430[2][-1:], r430[3], r430[4], r430[5], r430[6], r430[7], r430[8], r430[9], r430[10]))
                        i = i+1
                        prevRUL = r430[10]
source430.close()
result430.close()

# failure4
print('failure4 cycle determination is in progress...')
with open("data/01_intermediateData/PdM_08_RUL40_sorted_asc.csv", "r", newline='') as source440:
    reader440 = csv.reader(source440)
    with open("data/01_intermediateData/PdM_09_RUL_cycle_40.csv", "w", newline='') as result440:
        writer440 = csv.writer(result440)
        mIDprev = 1
        mID = 1
        i = 1
        prevRUL = 0
        for r440 in reader440:
            if r440[0] == "datetime":
                writer440.writerow((r440[0], r440[1], str('cycle'), r440[2], r440[3], r440[4], r440[5], r440[6], r440[7], r440[8], r440[9], r440[10]))
            else:
                mID = int(r440[1])
                if mID != mIDprev:
                    i = 1
                    writer440.writerow((r440[0], r440[1], str(i), r440[2][-1:], r440[3], r440[4], r440[5], r440[6], r440[7], r440[8], r440[9], r440[10]))
                    mIDprev = mID
                    i = i+1
                    prevRUL = r440[10]
                else:
                    if r440[9] != str(0):
                        writer440.writerow((r440[0], r440[1], str(i), r440[2][-1:], r440[3], r440[4], r440[5], r440[6], r440[7], r440[8], r440[9], r440[10]))
                        prevRUL = r440[10]
                    elif r440[9] == str(0) and prevRUL == str(0):
                        i = 1
                        writer440.writerow((r440[0], r440[1], str(i), r440[2][-1:], r440[3], r440[4], r440[5], r440[6], r440[7], r440[8], r440[9], r440[10]))
                        prevRUL = r440[10]
                        i = i+1
                    else:
                        writer440.writerow((r440[0], r440[1], str(i), r440[2][-1:], r440[3], r440[4], r440[5], r440[6], r440[7], r440[8], r440[9], r440[10]))
                        i = i+1
                        prevRUL = r440[10]
source440.close()
result440.close()

print('Writing of .../data/01_intermediateData/PdM_09_RUL_cycle_n.csv files completed successfully.')

Determine cycle is in progress...
failure1 cycle determination is in progress...
failure2 cycle determination is in progress...
failure3 cycle determination is in progress...
failure4 cycle determination is in progress...
Writing of .../data/01_intermediateData/PdM_09_RUL_cycle_n.csv files completed successfully.


In [12]:
# split data into train and test data
print('Split data into train and test data is in progress...')

# write train data for failure1
print('Writing train data for failure1 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_10.csv", "r", newline='') as source510:
    reader510 = csv.reader(source510)
    with open("data/01_intermediateData/PdM_10_train_10.csv", "w", newline='') as result510:
        writer510 = csv.writer(result510)
        for r510 in reader510:
            if r510[1] not in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer510.writerow(r510)
source510.close()
result510.close()

# write train data for failure2
print('Writing train data for failure2 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_20.csv", "r", newline='') as source520:
    reader520 = csv.reader(source520)
    with open("data/01_intermediateData/PdM_10_train_20.csv", "w", newline='') as result520:
        writer520 = csv.writer(result520)
        for r520 in reader520:
            if r520[1] not in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer520.writerow(r520)
source520.close()
result520.close()

# write train data for failure3
print('Writing train data for failure3 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_30.csv", "r", newline='') as source530:
    reader530 = csv.reader(source530)
    with open("data/01_intermediateData/PdM_10_train_30.csv", "w", newline='') as result530:
        writer530 = csv.writer(result530)
        for r530 in reader530:
            if r530[1] not in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer530.writerow(r530)
source530.close()
result530.close()

# write train data for failure4
print('Writing train data for failure4 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_40.csv", "r", newline='') as source540:
    reader540 = csv.reader(source540)
    with open("data/01_intermediateData/PdM_10_train_40.csv", "w", newline='') as result540:
        writer540 = csv.writer(result540)
        for r540 in reader540:
            if r540[1] not in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer540.writerow(r540)
source540.close()
result540.close()

# write test data for failure1
print('Writing test data for failure1 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_10.csv", "r", newline='') as source610:
    reader610 = csv.reader(source610)
    with open("data/01_intermediateData/PdM_10_test_10.csv", "w", newline='') as result610:
        writer610 = csv.writer(result610)
        for r610 in reader610:
            if r610[0] == "datetime" or r610[1] in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer610.writerow(r610)
source610.close()
result610.close()

# write test data for failure2
print('Writing test data for failure2 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_20.csv", "r", newline='') as source620:
    reader620 = csv.reader(source620)
    with open("data/01_intermediateData/PdM_10_test_20.csv", "w", newline='') as result620:
        writer620 = csv.writer(result620)
        for r620 in reader620:
            if r620[0] == "datetime" or r620[1] in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer620.writerow(r620)
source620.close()
result620.close()

# write test data for failure3
print('Writing test data for failure3 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_30.csv", "r", newline='') as source630:
    reader630 = csv.reader(source630)
    with open("data/01_intermediateData/PdM_10_test_30.csv", "w", newline='') as result630:
        writer630 = csv.writer(result630)
        for r630 in reader630:
            if r630[0] == "datetime" or r630[1] in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer630.writerow(r630)
source630.close()
result630.close()

# write test data for failure4
print('Writing test data for failure4 is in progress...')
with open("data/01_intermediateData/PdM_09_RUL_cycle_40.csv", "r", newline='') as source640:
    reader640 = csv.reader(source640)
    with open("data/01_intermediateData/PdM_10_test_40.csv", "w", newline='') as result640:
        writer640 = csv.writer(result640)
        for r640 in reader640:
            if r640[0] == "datetime" or r640[1] in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer640.writerow(r640)
source640.close()
result640.close()

print('Split data into train and test data completed successfully.')

Split data into train and test data is in progress...
Writing train data for failure1 is in progress...
Writing train data for failure2 is in progress...
Writing train data for failure3 is in progress...
Writing train data for failure4 is in progress...
Writing test data for failure1 is in progress...
Writing test data for failure2 is in progress...
Writing test data for failure3 is in progress...
Writing test data for failure4 is in progress...
Split data into train and test data completed successfully.


In [13]:
# read training data 
train_df10 = pd.read_csv('./data/01_intermediateData/PdM_10_train_10.csv', sep=",", header=0)
train_df20 = pd.read_csv('./data/01_intermediateData/PdM_10_train_20.csv', sep=",", header=0)
train_df30 = pd.read_csv('./data/01_intermediateData/PdM_10_train_30.csv', sep=",", header=0)
train_df40 = pd.read_csv('./data/01_intermediateData/PdM_10_train_40.csv', sep=",", header=0)
#train_df.head()

In [14]:
# read test data
test_df10 = pd.read_csv('./data/01_intermediateData/PdM_10_test_10.csv', sep=",", header=0)
test_df20 = pd.read_csv('./data/01_intermediateData/PdM_10_test_20.csv', sep=",", header=0)
test_df30 = pd.read_csv('./data/01_intermediateData/PdM_10_test_30.csv', sep=",", header=0)
test_df40 = pd.read_csv('./data/01_intermediateData/PdM_10_test_40.csv', sep=",", header=0)
#test_df.head()

## Data Preprocessing

In [15]:
# generate label columns for training data
w1 = 30
w0 = 15

# Label1 indicates a failure will occur within the next 30 cycles.
# 1 indicates failure, 0 indicates healthy 
train_df10['label1'] = np.where(train_df10['RUL'] <= w1, 1, 0 )
train_df20['label1'] = np.where(train_df20['RUL'] <= w1, 1, 0 )
train_df30['label1'] = np.where(train_df30['RUL'] <= w1, 1, 0 )
train_df40['label1'] = np.where(train_df40['RUL'] <= w1, 1, 0 )

# label2 is multiclass, value 1 is identical to label1,
# value 2 indicates failure within 15 cycles
train_df10['label2'] = train_df10['label1']
train_df10.loc[train_df10['RUL'] <= w0, 'label2'] = 2
train_df20['label2'] = train_df20['label1']
train_df20.loc[train_df20['RUL'] <= w0, 'label2'] = 2
train_df30['label2'] = train_df30['label1']
train_df30.loc[train_df30['RUL'] <= w0, 'label2'] = 2
train_df40['label2'] = train_df40['label1']
train_df40.loc[train_df40['RUL'] <= w0, 'label2'] = 2
#train_df.head()

In [16]:
# MinMax normalizatio - train - step 1
train_df10['cycle_norm'] = train_df10['cycle']
train_df10.drop(train_df10.columns[[9]], axis=1, inplace=True)
#train_df10.drop(train_df10.columns[[9]], axis=1, inplace=True)
train_df20['cycle_norm'] = train_df20['cycle']
train_df20.drop(train_df20.columns[[9]], axis=1, inplace=True)
#train_df20.drop(train_df20.columns[[9]], axis=1, inplace=True)
train_df30['cycle_norm'] = train_df30['cycle']
train_df30.drop(train_df30.columns[[9]], axis=1, inplace=True)
#train_df30.drop(train_df30.columns[[9]], axis=1, inplace=True)
train_df40['cycle_norm'] = train_df40['cycle']
train_df40.drop(train_df40.columns[[9]], axis=1, inplace=True)
#train_df40.drop(train_df40.columns[[9]], axis=1, inplace=True)

#train_df.head()

In [18]:
# MinMax normalization - train - step 2
# failure1
train_df_datetime10 = train_df10['datetime']
train_df10 = train_df10[['machineID','cycle','model','age','volt','rotate','pressure','vibration','failures','RUL','label1','label2','cycle_norm']]
cols_normalize10 = train_df10.columns.difference(['machineID','cycle','failures','RUL','label1','label2'])
min_max_scaler10 = MinMaxScaler()
norm_train_df10 = pd.DataFrame(min_max_scaler10.fit_transform(train_df10[cols_normalize10]), 
                             columns=cols_normalize10, 
                             index=train_df10.index)
join_df10 = train_df10[train_df10.columns.difference(cols_normalize10)].join(norm_train_df10)
train_df10 = join_df10.reindex(columns = train_df10.columns)
train_df10 = pd.concat([train_df_datetime10, train_df10],axis=1)

# failure2
train_df_datetime20 = train_df20['datetime']
train_df20 = train_df20[['machineID','cycle','model','age','volt','rotate','pressure','vibration','failures','RUL','label1','label2','cycle_norm']]
cols_normalize20 = train_df20.columns.difference(['machineID','cycle','failures','RUL','label1','label2'])
min_max_scaler20 = MinMaxScaler()
norm_train_df20 = pd.DataFrame(min_max_scaler20.fit_transform(train_df20[cols_normalize20]), 
                             columns=cols_normalize20, 
                             index=train_df20.index)
join_df20 = train_df20[train_df20.columns.difference(cols_normalize20)].join(norm_train_df20)
train_df20 = join_df20.reindex(columns = train_df20.columns)
train_df20 = pd.concat([train_df_datetime20, train_df20],axis=1)

# failure3
train_df_datetime30 = train_df30['datetime']
train_df30 = train_df30[['machineID','cycle','model','age','volt','rotate','pressure','vibration','failures','RUL','label1','label2','cycle_norm']]
cols_normalize30 = train_df30.columns.difference(['machineID','cycle','failures','RUL','label1','label2'])
min_max_scaler30 = MinMaxScaler()
norm_train_df30 = pd.DataFrame(min_max_scaler30.fit_transform(train_df30[cols_normalize30]), 
                             columns=cols_normalize30, 
                             index=train_df30.index)
join_df30 = train_df30[train_df30.columns.difference(cols_normalize30)].join(norm_train_df30)
train_df30 = join_df30.reindex(columns = train_df30.columns)
train_df30 = pd.concat([train_df_datetime30, train_df30],axis=1)

# failure4
train_df_datetime40 = train_df40['datetime']
train_df40 = train_df40[['machineID','cycle','model','age','volt','rotate','pressure','vibration','failures','RUL','label1','label2','cycle_norm']]
cols_normalize40 = train_df40.columns.difference(['machineID','cycle','failures','RUL','label1','label2'])
min_max_scaler40 = MinMaxScaler()
norm_train_df40 = pd.DataFrame(min_max_scaler40.fit_transform(train_df40[cols_normalize40]), 
                             columns=cols_normalize40, 
                             index=train_df40.index)
join_df40 = train_df40[train_df40.columns.difference(cols_normalize40)].join(norm_train_df40)
train_df40 = join_df40.reindex(columns = train_df40.columns)
train_df40 = pd.concat([train_df_datetime40, train_df40],axis=1)

#train_df.head()

In [19]:
# MinMax normalizatio - test - step 1
# failure1
test_df10['cycle_norm'] = test_df10['cycle']
test_df10.drop(test_df10.columns[[0]], axis=1, inplace=True)
test_df10.drop(test_df10.columns[[8]], axis=1, inplace=True)
test_df10.drop(test_df10.columns[[8]], axis=1, inplace=True)

# failure2
test_df20['cycle_norm'] = test_df20['cycle']
test_df20.drop(test_df20.columns[[0]], axis=1, inplace=True)
test_df20.drop(test_df20.columns[[8]], axis=1, inplace=True)
test_df20.drop(test_df20.columns[[8]], axis=1, inplace=True)

# failure3
test_df30['cycle_norm'] = test_df30['cycle']
test_df30.drop(test_df30.columns[[0]], axis=1, inplace=True)
test_df30.drop(test_df30.columns[[8]], axis=1, inplace=True)
test_df30.drop(test_df30.columns[[8]], axis=1, inplace=True)

# failure4
test_df40['cycle_norm'] = test_df40['cycle']
test_df40.drop(test_df40.columns[[0]], axis=1, inplace=True)
test_df40.drop(test_df40.columns[[8]], axis=1, inplace=True)
test_df40.drop(test_df40.columns[[8]], axis=1, inplace=True)

#test_df.head()

In [20]:
# MinMax normalizatio - test - step 2
# failure1
cols_normalize10 = test_df10.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler10 = MinMaxScaler()
norm_test_df10 = pd.DataFrame(min_max_scaler10.fit_transform(test_df10[cols_normalize10]), 
                             columns=cols_normalize10, 
                             index=test_df10.index)
join_df10 = test_df10[test_df10.columns.difference(cols_normalize10)].join(norm_test_df10)
test_df10 = join_df10.reindex(columns = test_df10.columns)

# failure2
cols_normalize20 = test_df20.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler20 = MinMaxScaler()
norm_test_df20 = pd.DataFrame(min_max_scaler20.fit_transform(test_df20[cols_normalize20]), 
                             columns=cols_normalize20, 
                             index=test_df20.index)
join_df20 = test_df20[test_df20.columns.difference(cols_normalize20)].join(norm_test_df20)
test_df20 = join_df20.reindex(columns = test_df20.columns)

# failure3
cols_normalize30 = test_df30.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler30 = MinMaxScaler()
norm_test_df30 = pd.DataFrame(min_max_scaler30.fit_transform(test_df30[cols_normalize30]), 
                             columns=cols_normalize30, 
                             index=test_df30.index)
join_df30 = test_df30[test_df30.columns.difference(cols_normalize30)].join(norm_test_df30)
test_df30 = join_df30.reindex(columns = test_df30.columns)

# failure4
cols_normalize40 = test_df40.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler40 = MinMaxScaler()
norm_test_df40 = pd.DataFrame(min_max_scaler40.fit_transform(test_df40[cols_normalize40]), 
                             columns=cols_normalize40, 
                             index=test_df40.index)
join_df40 = test_df40[test_df40.columns.difference(cols_normalize40)].join(norm_test_df40)
test_df40 = join_df40.reindex(columns = test_df40.columns)

#test_df.head()

In [21]:
# generate label columns w0 and w1 for test data
# failure1
test_df10['label1'] = np.where(test_df10['RUL'] <= w1, 1, 0 )
test_df10['label2'] = test_df10['label1']
test_df10.loc[test_df10['RUL'] <= w0, 'label2'] = 2

# failure2
test_df20['label1'] = np.where(test_df20['RUL'] <= w1, 1, 0 )
test_df20['label2'] = test_df20['label1']
test_df20.loc[test_df20['RUL'] <= w0, 'label2'] = 2

# failure3
test_df30['label1'] = np.where(test_df30['RUL'] <= w1, 1, 0 )
test_df30['label2'] = test_df30['label1']
test_df30.loc[test_df30['RUL'] <= w0, 'label2'] = 2

# failure4
test_df40['label1'] = np.where(test_df40['RUL'] <= w1, 1, 0 )
test_df40['label2'] = test_df40['label1']
test_df40.loc[test_df40['RUL'] <= w0, 'label2'] = 2

#test_df.head()

## Data Visualization

In [22]:
# preparing data for visualizations 
# window of 50 cycles prior to a failure point for engine id 89
engine_id3 = test_df10[test_df10['machineID'] == 89]
#engine_id3 = test_df20[test_df20['machineID'] == 89]
#engine_id3 = test_df30[test_df30['machineID'] == 89]
#engine_id3 = test_df40[test_df40['machineID'] == 89]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
cols1 = ['volt', 'rotate']
engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
cols2 = ['pressure', 'vibration']
engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

# plotting sensor data for engine ID 89 prior to a failure point - sensors 1-2 
ax1 = engine_id3_50cycleWindow1.plot(subplots=True, sharex=True, figsize=(20,20))

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [ ]:
# plotting sensor data for engine ID 89 prior to a failure point - sensors 3-4 
ax2 = engine_id3_50cycleWindow2.plot(subplots=True, sharex=True, figsize=(20,20))

## Persist the data sets

In [23]:
# The data was read in using a Pandas data frame. We'll convert store it for later manipulations in subsequent notebooks.
print('Data saving into files in progress...')

train_df10.to_csv('./data/02_preparedData/PdM_train_prepared_failure1.csv', encoding='utf-8', index=False)
train_df20.to_csv('./data/02_preparedData/PdM_train_prepared_failure2.csv', encoding='utf-8', index=False)
train_df30.to_csv('./data/02_preparedData/PdM_train_prepared_failure3.csv', encoding='utf-8', index=False)
train_df40.to_csv('./data/02_preparedData/PdM_train_prepared_failure4.csv', encoding='utf-8', index=False)

test_df10.to_csv('./data/02_preparedData/PdM_test_prepared_failure1.csv', encoding='utf-8', index=False)
test_df20.to_csv('./data/02_preparedData/PdM_test_prepared_failure2.csv', encoding='utf-8', index=False)
test_df30.to_csv('./data/02_preparedData/PdM_test_prepared_failure3.csv', encoding='utf-8', index=False)
test_df40.to_csv('./data/02_preparedData/PdM_test_prepared_failure4.csv', encoding='utf-8', index=False)

print("Data files saved!")

Data saving into files in progress...
Data files saved!
